In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

#AWS
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")

In [2]:
#Listar arquivos do bucket do S3 a partir de um prefixo

import boto3

session = boto3.Session( 
         aws_access_key_id=aws_access_key_id, 
         aws_secret_access_key=aws_secret_access_key)

#Then use the session to get the resource
s3 = session.resource('s3')

my_bucket = s3.Bucket('saks-raw-data')

CFDFinanceAmounts = []
for objects in my_bucket.objects.filter(Prefix='saxobank/CFDFinanceAmounts/'):
    CFDFinanceAmounts.append(objects.key)

In [4]:
CFDFinanceAmounts

['saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01022023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01032023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01052023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01062023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01112022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01122022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02012023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02022023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02032023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02052023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02062023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02112022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02122022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_03012023.txt',
 'saxobank/CFDFinanc

In [3]:
#Conexão ao S3 - CLIENT

s3_client = boto3.client("s3",
aws_access_key_id=aws_access_key_id,
aws_secret_access_key=aws_secret_access_key
)

In [ ]:
#le varios arquivos de um bucket do s3, colocando-os em um dicionário
import boto3
import io

arqs = {}
for objeto in elementos_selecionados:
    print(objeto)
    obj = s3_client.get_object(Bucket='saks-raw-data', Key='{key}'.format(key=objeto))
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), delimiter=';', encoding='latin1')
    arqs.update({objeto.replace('.txt','').replace('saxobank/PLConversionRates/',''):df})

In [ ]:
#transforma os arquivos e os guarda em outro dicionario

arqs_transform = {}

for i in arqs:
    df = arqs['{}'.format(i)]
    df['TradeDate'] = df['TradeDate'].astype(str)
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])

    arqs_transform.update({i: df})

In [ ]:
#envia os arquivos transformados para o s3
for i in arqs_transform:
    arqs_transform['{}'.format(i)].to_csv('s3://saks-clean-data/saxobank/PLConversionRates/{}.csv'.format(i), index=False)

In [ ]:
# apagar objetos de um bucket
import boto3

def delete_s3_objects(bucket_name, object_keys):
    # Cria uma instância do cliente do S3
    s3 = boto3.client('s3')

    # Exclui os objetos do bucket
    response = s3.delete_objects(
        Bucket=bucket_name,
        Delete={
            'Objects': [{'Key': key} for key in object_keys]
        }
    )

    # Verifica se a exclusão foi bem-sucedida
    if 'Deleted' in response:
        print("Objetos excluídos com sucesso:")
        for deleted_object in response['Deleted']:
            print(deleted_object['Key'])
    else:
        print("Falha ao excluir objetos do bucket.")

# Exemplo de uso
bucket_name = 'saks-raw-data'
object_keys = ['objeto1.txt', 'objeto2.jpg']  # Lista de chaves dos objetos a serem excluídos

delete_s3_objects(bucket_name, object_keys)


In [23]:
#verifica objetos no glacier
import boto3
deep = []
# Nome do bucket
bucket_name = 'saks-raw-data'
prefix = 'saxobank/CFDFinanceAmounts/'

# Listar objetos no bucket com o prefixo especificado
bucket = s3.Bucket(bucket_name)
for obj in bucket.objects.filter(Prefix=prefix):
    # Verificar se o objeto está armazenado no Glacier
    storage_class = obj.storage_class
    if storage_class == 'DEEP_ARCHIVE':
        deep.append(obj.key)



In [24]:
deep

['saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01112022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_01122022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02012023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02112022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_02122022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_03012023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_03102022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_03112022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_04012023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_04102022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_04112022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_05012023.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_05102022.txt',
 'saxobank/CFDFinanceAmounts/CFDFinanceAmounts_5364997_05122022.txt',
 'saxobank/CFDFinanc

In [20]:
#restaurar um objeto do glacier
bucket_name = 'saks-raw-data'

for i in deep:
    s3_client.restore_object(
        Bucket=bucket_name,
        Key=i,
        RestoreRequest={'Days': 7}  # Tempo de restauração (em dias)
)


# ##verificar se o objeto foi restaurado
# response = s3_client.head_object(
#     Bucket=bucket_name,
#     Key='saxobank/PLConversionRates/PLConversionRates_5364997_03012022.txt'
# )

# status = response['Restore']
# if status.startswith('ongoing-request="false"'):
#     print("A restauração está completa. O arquivo está pronto para ser baixado.")
# else:
#     print("Ainda em andamento. Aguarde mais algum tempo.")

In [ ]:
#dowload file
import boto3
import os

# Criar um cliente S3
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Nome do bucket no S3
bucket_name = 'saks-raw-data'

# Diretório local para salvar os arquivos
local_directory = r"C:\Users\natal\OneDrive\Documentos\Saks\Receita\InterestAmounts\setembro"

# Iterar sobre os elementos selecionados e baixar cada arquivo
for objeto in elementos_selecionados:
    local_file_path = os.path.join(local_directory, os.path.basename(objeto))

    # Criar o diretório local se não existir
    os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

    # Baixar o arquivo do S3
    s3.download_file(bucket_name, objeto, local_file_path)

    print(f'O arquivo {objeto} foi baixado e salvo como {local_file_path}')

print('Download concluído!')
